In [1]:
# Import statements:
import pandas as pd
import numpy as np
import time
# Hugging face import:
from transformers import pipeline



2024-02-12 23:11:20.242825: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
theme_pipeline = pipeline("zero-shot-classification",
                      model="facebook/bart-large-mnli")
sentiment_pipeline = pipeline("sentiment-analysis")

No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


In [3]:
# Loading in data:
df = pd.read_csv("/Users/amaribauer/Desktop/A_ML/FinalProject/okcupid_profiles.csv")
#df = pd.read_csv("/Users/rohitkandala/Desktop/UChicago/Academic Quarters/2023-24/Winter 2024/CAPP 30255/Project/okcupid_profiles.csv")

In [5]:
df.isna().sum().sort_values(ascending=True)

age                0
location           0
last_online        0
income             0
sex                0
status             0
orientation        0
height             3
speaks            50
drinks          2985
body_type       5296
essay0          5488
smokes          5512
ethnicity       5680
education       6628
essay1          7572
job             8198
essay2          9638
essay4         10537
essay5         10850
sign           11056
essay3         11476
essay7         12451
essay9         12603
essay6         13771
drugs          14080
essay8         19225
pets           19921
religion       20226
diet           24395
offspring      35561
dtype: int64

essay0- My self summary
essay1- What I’m doing with my life
essay2- I’m really good at
essay3- The first thing people usually notice about me
essay4- Favorite books, movies, show, music, and food
essay5- The six things I could never do without
essay6- I spend a lot of time thinking about
essay7- On a typical Friday night I am
essay8- The most private thing I am willing to admit
essay9- You should message me if...

In [6]:
# Creating a dataframe with just the essays:
# I limited us to half the essays for computation reasons. We can expand back if we see the need
which_essays = ["essay0", "essay1", "essay2", "essay3",  "essay5"]
essays_df = df.loc[:,which_essays ]

In [ ]:
# Write cleaning function that removes stopwords, etc - potentially already done by pretrained model. 

Proposed Additions to labels: travel, drinking, drugs, kids
Proposed Removals: teen, enthusiastic, time periods, avid, miscellaneous, rock, sci-fi, favorite, novelty

In [7]:
candidate_labels = ['TV hits', 'enthusiastic', 'movies', 'music',
          'comedies', 'food', 'teen', 'everything', 'drama', 'time periods', 
          'avid', 'miscellaneous', 'rock', 'music', 'sci-fi', 'favorite', 
          'novelty', 'books']

In [8]:
essays_df

,essay0,essay1,essay2,essay3,essay5
0,about me: i would love to think that i was so...,currently working as an international agent fo...,making people laugh. ranting about a good salt...,"the way i look. i am a six foot half asian, ha...",food. water. cell phone. shelter.
1,i am a chef: this is what that means. 1. i am ...,dedicating everyday to being an unbelievable b...,being silly. having ridiculous amonts of fun w...,NaN,delicious porkness in all of its glories. my b...
2,"i'm not ashamed of much, but writing public te...","i make nerdy software for musicians, artists, ...",improvising in different contexts. alternating...,my large jaw and large glasses are the physica...,movement conversation creation contemplation t...
3,i work in a library and go to school. . .,reading things written by old dead people,playing synthesizers and organizing books acco...,socially awkward but i do my best,NaN
4,hey how's it going? currently vague on the pro...,work work work work + play,creating imagery to look at: http://bagsbrown....,i smile a lot and my inquisitive nature,NaN
...,...,...,...,...,...
59941,"vibrant, expressive, caring optimist. i love b...",the happiest times have been when life came to...,i make an outstanding osso bucco. i am also ve...,"i am told that people notice my smile, eyes an...","my family, my dog, italy, words and music!"
59942,i'm nick. i never know what to write about mys...,currently finishing school for film production...,"filmmaking, photography, graphic design, web d...","dude, i don't know.",iphone contact lenses headphones camera tv rem...
59943,"hello! i enjoy traveling, watching movies, and...","i'm a civil engineer, who enjoys helping the c...",- looking at things objectively - getting thin...,i'm quiet until i get used to the environment ...,- iphone - friends and family - internet - bay...
59944,"""all i have in this world are my balls and my ...","following my dreams... ""you got a dream... you...",listening,it used to be the hair until i mowed it off bu...,"music, family, friends, a basketball, hoop, so..."


In [9]:
#I am applying the thematic classifier to the first row of the dataframe.
#This outputs a list of dictionaries with the sequence, labels, and scores keys
results = essays_df.iloc[0].apply(lambda essay: theme_pipeline(essay, candidate_labels))

In [13]:

results[0] 

{'sequence': "about me:  i would love to think that i was some some kind of intellectual: either the dumbest smart guy, or the smartest dumb guy. can't say i can tell the difference. i love to talk about ideas and concepts. i forge odd metaphors instead of reciting cliches. like the simularities between a friend of mine's house and an underwater salt mine. my favorite word is salt by the way (weird choice i know). to me most things in life are better as metaphors. i seek to make myself a little better everyday, in some productively lazy way. got tired of tying my shoes. considered hiring a five year old, but would probably have to tie both of our shoes... decided to only wear leather shoes dress shoes.  about you:  you love to have really serious, really deep conversations about really silly stuff. you have to be willing to snap me out of a light hearted rant with a kiss. you don't have to be funny, but you have to be able to make me laugh. you should be able to bend spoons with your m

In [21]:
#I am applying the sentiment model to each essay in the first row.
#For each essay, we'll have a top theme and sentiment result
sentiments = pd.DataFrame(results.apply(lambda essay: (results.name, essay['sequence'],essay['labels'][0], sentiment_pipeline(essay['sequence']))))                          
sentiments[0][0]

(0,
 "about me:  i would love to think that i was some some kind of intellectual: either the dumbest smart guy, or the smartest dumb guy. can't say i can tell the difference. i love to talk about ideas and concepts. i forge odd metaphors instead of reciting cliches. like the simularities between a friend of mine's house and an underwater salt mine. my favorite word is salt by the way (weird choice i know). to me most things in life are better as metaphors. i seek to make myself a little better everyday, in some productively lazy way. got tired of tying my shoes. considered hiring a five year old, but would probably have to tie both of our shoes... decided to only wear leather shoes dress shoes.  about you:  you love to have really serious, really deep conversations about really silly stuff. you have to be willing to snap me out of a light hearted rant with a kiss. you don't have to be funny, but you have to be able to make me laugh. you should be able to bend spoons with your mind, and

In [22]:
#TO DO: BUILD OUT LOGIC TO PROCESS MORE ROWS

# Apply the sentiment_pipeline function to each essay in the results Series and transpose the resulting DataFrame
sentiments_df = pd.DataFrame(results.apply(lambda essay: [results.name, 
        essay['sequence'], essay['labels'][0], sentiment_pipeline(essay['sequence'])]).tolist(
        ), columns=['Index', 'Sequence', 'Label', 'Sentiment']).transpose()

# Combine all columns into a single row
long_col_names =  which_essays + ['theme1', 'theme2','theme3', 'theme4', 'theme5'] + ['sent1','sent2','sent3','sent4','sent5']
sentiments_combined = pd.DataFrame(sentiments_df.values.flatten().reshape(1, -1))

Important: Our end result here, sentiments_long, is a pandas df with the essay responses, the top themes, and sentiment analysis response for five essays. Our output for the eventual final classifier would a pandas dataframe with this information for every profile. An alternative implementation could be to include each essay, top theme, and sentiment into a tuple, and then we'd only have five columns per person instead. 

In [23]:
sentiments_long = sentiments_combined.drop(sentiments_combined.columns[:5], axis = 1)
sentiments_long.columns = long_col_names
sentiments_long

,essay0,essay1,essay2,essay3,essay5,theme1,theme2,theme3,theme4,theme5,sent1,sent2,sent3,sent4,sent5
0,about me: i would love to think that i was so...,currently working as an international agent fo...,making people laugh. ranting about a good salt...,"the way i look. i am a six foot half asian, ha...",food. water. cell phone. shelter.,favorite,books,enthusiastic,miscellaneous,food,"[{'label': 'POSITIVE', 'score': 0.997051715850...","[{'label': 'NEGATIVE', 'score': 0.997330904006...","[{'label': 'POSITIVE', 'score': 0.998056292533...","[{'label': 'NEGATIVE', 'score': 0.853669345378...","[{'label': 'NEGATIVE', 'score': 0.740077495574..."


Next steps:
- Saving probabilities into a dataset after computing all of them, consult with Ethan on the topic names prior to running everything

----

Generative AI portion:

- import gpt-2 or some other model
- fine tune model on the essays in the dataset
- generate text in response to the input essay

link with example of gpt2: https://huggingface.co/openai-community/gpt2

- How do we incorporate higher matches to train the model? ## Ask the TA? 
    - One model is most likely feasible, but the best we have brain stormed is what if we do multiple models (one for each label)
        - When user inputs text, classifier identifies top topic -> generate text in response with corresponding topic model

-----

Matching methodology:

- One proposal: lets take the probabilities for each label between two individuals and compute the distance between the probabilities across ALL categories and aggregate them, that is the "compatiability index". 
    - Only conduct matching search for those who are compatiable, sexuality-wise
        - Goal is to limit the amount of cross-computation
    - Compute compatiability index ONLY between people who have the same top topic AND sexualtiy

-----

